In [ ]:
# ==============================================================================
# Colab Notebook: Model Optimization and Benchmarking
#
# This script takes a pre-trained Keras model and its corresponding assets,
# then creates and benchmarks optimized TFLite versions (INT8 and Float16)
# for Android deployment.
#
# It is designed to run AFTER 'TCNonOwnData_v2.ipynb' has completed.
#
# Workflow:
# 1. Load the Keras model and reconstruct scaler/encoder from JSON files.
# 2. Re-create the exact same data splits for fair evaluation.
# 3. Convert the model to a highly-optimized INT8 TFLite model.
# 4. Convert the model to a high-accuracy Float16 TFLite model.
# 5. Save all new TFLite models and a copy of the JSONs to a new
#    'Converted_Assets' subdirectory.
# 6. Run a comprehensive benchmark measuring accuracy, latency, and size
#    for all three model versions (Keras, INT8, Float16).
# 7. Provide a final recommendation based on the results.
# ==============================================================================

# --- Block 1: Setup and Configuration ---
print("--- Block 1: Setup and Configuration ---")
import os
import time
import json
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Path Configuration ---
# 1. Point to the results directory from your V2 training script
BASE_RESULTS_DIR = Path('/content/drive/MyDrive/Colab_HAR_Project/results/TCN_Results_v2_with_Assets')
FILE_PREFIX = "tcn_v2_ABCDE_"

# 2. Define the new subdirectory for our converted assets
CONVERTED_ASSETS_DIR = BASE_RESULTS_DIR / 'Converted_Assets'
CONVERTED_ASSETS_DIR.mkdir(parents=True, exist_ok=True) # Create it
print(f"All new assets will be saved to: {CONVERTED_ASSETS_DIR}")

# 3. Define paths to SOURCE assets (from the training run)
KERAS_MODEL_PATH = BASE_RESULTS_DIR / f'{FILE_PREFIX}har_model.keras'
SCALER_PATH_JSON_SRC = BASE_RESULTS_DIR / f'{FILE_PREFIX}scaler.json'
LABELS_PATH_JSON_SRC = BASE_RESULTS_DIR / f'{FILE_PREFIX}labels.json'

# 4. Define paths for the new DESTINATION assets
TFLITE_INT8_PATH_DEST = CONVERTED_ASSETS_DIR / 'har_model_int8.tflite'
TFLITE_FLOAT16_PATH_DEST = CONVERTED_ASSETS_DIR / 'har_model_float16.tflite'
SCALER_PATH_JSON_DEST = CONVERTED_ASSETS_DIR / 'scaler.json'
LABELS_PATH_JSON_DEST = CONVERTED_ASSETS_DIR / 'labels.json'

# ==============================================================================
# Block 2: Load Data and Reconstruct Assets from JSON (Corrected and Final)
# ==============================================================================
print("\n--- Block 2: Load Data and Reconstruct Assets from JSON ---")

# --- Function to recreate data windows (copied from training script) ---
def create_subject_activity_windows(df, window_size, stride):
    windows, labels = [], []
    required_cols = ['x_accel', 'y_accel', 'z_accel', 'x_gyro', 'y_gyro', 'z_gyro']
    for _, group_df in df.groupby(['subject', 'activity']):
        data_values = group_df.sort_values('timestamp')[required_cols].values
        if len(data_values) < window_size: continue
        for start in range(0, len(data_values) - window_size + 1, stride):
            windows.append(data_values[start : start + window_size])
            labels.append(group_df["activity"].iloc[0])
    return np.array(windows), np.array(labels)

# --- Recreate the exact same Train/Test split ---
input_data_path = Path('/content/drive/MyDrive/Colab_HAR_Project/data')
df1 = pd.read_csv(input_data_path / 'resampled_normalized_phone_data.csv')
df2 = pd.read_csv(input_data_path / 'combined_collected_data.csv')
df1_f = df1[df1['activity'].isin(['B', 'D', 'E'])]
df2_f = df2[df2['activity'].isin(['A', 'C'])]
combined_df = pd.concat([df1_f, df2_f], ignore_index=True)

X, y_raw = create_subject_activity_windows(combined_df, 60, 15)

# --- Reconstruct LabelEncoder and Scaler from their JSON files ---
with open(LABELS_PATH_JSON_SRC) as f:
    labels_dict = json.load(f)
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(list(labels_dict.values()))
y = label_encoder.transform(y_raw)
print(f"Label Encoder reconstructed. Classes: {list(label_encoder.classes_)}")

with open(SCALER_PATH_JSON_SRC) as f:
    scaler_params = json.load(f)
scaler = StandardScaler()
scaler.mean_ = np.array(scaler_params['mean'])
scaler.scale_ = np.array(scaler_params['scale'])
print("Scaler object reconstructed from JSON file.")

# --- Perform the consistent data split and scaling ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y)
X_train_scaled = scaler.transform(X_train.reshape(-1, 6)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 6)).reshape(X_test.shape)

# ==============================================================================
# --- THE FIX IS HERE ---
# Explicitly cast the scaled data to float32 to match TensorFlow's expectations.
X_train_scaled = X_train_scaled.astype(np.float32)
X_test_scaled = X_test_scaled.astype(np.float32)
print(f"Data types corrected. X_test_scaled dtype is now: {X_test_scaled.dtype}")
# ==============================================================================

# --- Load the pre-trained Keras model ---
keras_model = tf.keras.models.load_model(KERAS_MODEL_PATH)
print("Keras model loaded successfully.")

# ==============================================================================
# --- Block 3: TFLite Model Conversions ---
# ==============================================================================
print("\n--- Block 3: TFLite Model Conversions ---")

# --- Strategy 1: Convert to INT8 with a larger representative dataset ---
print("  [1/2] Converting to INT8...")
def representative_dataset_gen():
    num_samples = min(1000, len(X_train_scaled))
    for i in np.random.choice(len(X_train_scaled), num_samples, replace=False):
        yield [X_train_scaled[i:i+1].astype(np.float32)]

converter_int8 = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]
converter_int8.representative_dataset = representative_dataset_gen
converter_int8.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_int8.inference_input_type = tf.int8
converter_int8.inference_output_type = tf.int8
tflite_int8_model = converter_int8.convert()
with open(TFLITE_INT8_PATH_DEST, 'wb') as f:
    f.write(tflite_int8_model)
print(f"  INT8 model saved successfully.")

# --- Strategy 2: Convert to Float16 ---
print("  [2/2] Converting to Float16...")
converter_float16 = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter_float16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_float16.target_spec.supported_types = [tf.float16]
tflite_float16_model = converter_float16.convert()
with open(TFLITE_FLOAT16_PATH_DEST, 'wb') as f:
    f.write(tflite_float16_model)
print(f"  Float16 model saved successfully.")

# --- Copy the JSON files to the new directory for a complete package ---
shutil.copy(SCALER_PATH_JSON_SRC, SCALER_PATH_JSON_DEST)
shutil.copy(LABELS_PATH_JSON_SRC, LABELS_PATH_JSON_DEST)
print("  JSON assets copied to the new directory.")


# ==============================================================================
# --- Block 4: Benchmarking and Final Report ---
# ==============================================================================
print("\n--- Block 4: Benchmarking and Final Report ---")

def benchmark_model(model_type, model_path=None, keras_model_instance=None):
    """Measures accuracy and inference speed for a given Keras or TFLite model."""
    print(f"  Benchmarking {model_type} model...")
    # Keras Model
    if model_type == 'Keras':
        y_pred = np.argmax(keras_model_instance.predict(X_test_scaled, verbose=0), axis=1)
        accuracy = accuracy_score(y_test, y_pred)
        latencies = []
        for i in range(100):
            if i >= 10: start_time = time.perf_counter()
            _ = keras_model_instance.predict(X_test_scaled[i:i+1], verbose=0)
            if i >= 10: latencies.append((time.perf_counter() - start_time) * 1000)
        return accuracy, np.mean(latencies)

    # TFLite Models
    interpreter = tf.lite.Interpreter(model_path=str(model_path))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    y_pred = []
    is_int8 = input_details['dtype'] == np.int8

    for i in range(len(X_test_scaled)):
        sample = X_test_scaled[i:i+1]
        if is_int8:
            scale, zero_point = input_details['quantization']
            sample = (sample / scale + zero_point).astype(np.int8)
        interpreter.set_tensor(input_details['index'], sample)
        interpreter.invoke()
        y_pred.append(np.argmax(interpreter.get_tensor(output_details['index'])[0]))
    accuracy = accuracy_score(y_test, y_pred)

    latencies = []
    for i in range(100):
        sample = X_test_scaled[i:i+1]
        if is_int8:
            scale, zero_point = input_details['quantization']
            sample = (sample / scale + zero_point).astype(np.int8)
        if i >= 10: start_time = time.perf_counter()
        interpreter.set_tensor(input_details['index'], sample)
        interpreter.invoke()
        if i >= 10: latencies.append((time.perf_counter() - start_time) * 1000)
    return accuracy, np.mean(latencies)

# --- Run benchmarks ---
keras_acc, keras_speed = benchmark_model('Keras', keras_model_instance=keras_model)
int8_acc, int8_speed = benchmark_model('TFLite INT8', model_path=TFLITE_INT8_PATH_DEST)
float16_acc, float16_speed = benchmark_model('TFLite Float16', model_path=TFLITE_FLOAT16_PATH_DEST)

# --- Prepare and Print Final Report ---
report_data = {
    "Model Type": ["Keras (Float32)", "TFLite (INT8)", "TFLite (Float16)"],
    "Accuracy": [f"{keras_acc:.4f}", f"{int8_acc:.4f}", f"{float16_acc:.4f}"],
    "Avg. Latency (ms)": [f"{keras_speed:.3f}", f"{int8_speed:.3f}", f"{float16_speed:.3f}"],
    "Size (KB)": [
        f"{os.path.getsize(KERAS_MODEL_PATH) / 1024:.1f}",
        f"{os.path.getsize(TFLITE_INT8_PATH_DEST) / 1024:.1f}",
        f"{os.path.getsize(TFLITE_FLOAT16_PATH_DEST) / 1024:.1f}"
    ]
}
report_df = pd.DataFrame(report_data)

print("\n" + "="*70)
print("             MODEL OPTIMIZATION AND BENCHMARK REPORT")
print("="*70)
print(report_df.to_string(index=False))
print("="*70)

# --- Provide a final recommendation ---
print("\nRecommendation:")
if int8_acc >= 0.985 and int8_speed < float16_speed:
    print("✅ The INT8 model is the clear winner. It maintains high accuracy while being the smallest and fastest.")
elif float16_acc > int8_acc + 0.01:
     print("✅ The Float16 model is the recommended choice. It significantly improves accuracy over INT8, making it worth the slight size/speed trade-off.")
else:
    print("⚠️ The INT8 model offers the best speed/size but has a noticeable accuracy drop. The Float16 model is a safer choice if accuracy is the top priority.")
    print("   For your Android app, test both to see which feels better in practice.")

print("\nFinal deployable assets are located in:")
print(CONVERTED_ASSETS_DIR)
!ls -lh {CONVERTED_ASSETS_DIR}

--- Block 1: Setup and Configuration ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All new assets will be saved to: /content/drive/MyDrive/Colab_HAR_Project/results/TCN_Results_v2_with_Assets/Converted_Assets

--- Block 2: Load Data and Reconstruct Assets from JSON ---
Label Encoder reconstructed. Classes: [np.str_('A'), np.str_('B'), np.str_('C'), np.str_('D'), np.str_('E')]
Scaler object reconstructed from JSON file.
Data types corrected. X_test_scaled dtype is now: float32
Keras model loaded successfully.

--- Block 3: TFLite Model Conversions ---
  [1/2] Converting to INT8...
Saved artifact at '/tmp/tmph8ladoam'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 6), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  138991927626064: TensorSpec(shape=(), dtype=tf.

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


  INT8 model saved successfully.
  [2/2] Converting to Float16...
Saved artifact at '/tmp/tmptnmw2cg_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 6), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  138991927626064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927624912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927627984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927624720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927624336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927627600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927625680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927629136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927623760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138991927627792: TensorSp